In [ ]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import ticker


import sys, warnings, glob
warnings.filterwarnings("once", category=RuntimeWarning)

## The function below performs smoothing on power spectra in frequency space

In [ ]:
##############################################################
### Function to smooth out ripples in data in frequency space
def freq_smooth2(freq,psd,winfac=1):
    """
    Required inputs:
    freq   (float 1D array) = frequencies (ascending) [Hz]
    psd    (float 1D or 2D array) = corresponding power spectral density [v**2/Hz]
    
    Optional inputs:
    winfac          (float) = scaling factor for window size as function of frequency
    
    Output:
    smooth (float 1D or 2D array) = Smoothed version of psd
    """
    
    smooth = psd.copy()
    if freq.ndim != 1:
        raise IndexError("Frequencies must be in a 1D array")
    win = np.rint(np.exp(np.sqrt(freq)*winfac))*2 - 1 # Window width
    
    if len(psd.shape) == 1:  # 1D case - easy:  
        if freq.shape != psd.shape:
            raise IndexError("Required input series are not the same length")
        for n in range(len(freq)):
            t0 = int((win[n]-1)/2)
            t1 = np.min((int((win[n]-1)/2)+1,len(freq)))
            smooth[n] = psd[n-t0:n+t1].mean()
            
    elif len(psd.shape) == 2:  # 2D case - harder:
        if q_lmean.wavelength.size not in psd.shape:
            raise IndexError("No PSD array dimension matches frequency series length")
        idx = psd.shape.index(freq.size) # This is the matching dimension for frequencies
        for n in range(len(freq)):
            t0 = int((win[n]-1)/2)
            t1 = np.min((int((win[n]-1)/2)+1,len(freq)))
            for j in range(psd.shape[1-idx]):
                if idx == 1:
                    smooth[:,n] = psd[:,n-t0:n+t1].mean(axis=idx)   
                else:
                    smooth[n,:] = psd[n-t0:n+t1,:].mean(axis=idx)   
        
    else:
        raise IndexError("Required PSD array cannot exceed 2D")
        
    return smooth

### Functions for custom color maps

In [ ]:
##############################################################
### Gradient Maker code

import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors


def hex_to_rgb(hex_string):
    '''
    Converts hex string to rgb colors
    
    Required input:
        hex_string (str) = String of characters representing a hex color.
                           May or ay not have a leading `#`
                           Any alpha value on the end will be stripped off
    Output:
        A list (length 3) of integers (range 0-255) of RGB values
    '''
    value = hex_string.strip("#")[:6] # removes hash symbol if present, alpha value on end
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))


def rgb_to_hex(tup3):
    '''
    Converts rgb 3-member list or array to hex colors
    
    Required input: 
        tup3 (tuple) = list (length 3) of RGB values in decimal 0-255 range   
        
    Output: 
        A String of 6 characters representing a hex color, with leading hash.
    '''
    return "#{:02x}{:02x}{:02x}".format(tup3[0],tup3[1],tup3[2])


def rgb_to_dec(rgb_list):
    '''
    Converts RGB (range 0-255) to decimal (range 0-1) colors (i.e. divides each value by 255)
    
    Required input:
        rgb_list (list): list (length 3) of RGB values (range 0-255)
        
    Output:
        A list (length 3) of decimal values (0-1)
    '''
    return [v/255 for v in rgb_list[:3]]


def dec_to_rgb(dec_list):
    '''
    Converts decimal (range 0-1) to RGB (range 0-255) colors
    
    Required input:
        dec_list (list): list (length 3) of decimal values (0-1)
        
    Output:
        A list (length 3) of RGB values (range 0-255)
    '''
    return [int(round(v*255)) for v in dec_list[:3]]


def get_continuous_cmap(hex_list, float_list=None, ncol=256):
    ''' 
    Creates and returns a color map that can be used in heat map figures.
        The parameter `float_list` can be used to control the spacing between colors:
            If float_list is not provided, color map graduates linearly between each color in hex_list.
            If float_list is provided, each color in hex_list is mapped to the respective relative location in float_list. 
        
    Required input:
        hex_list   (list): List of hex code strings
        
    Optional input:
        float_list (list): List of floats between 0 and 1, same length as hex_list. Must start with 0 and end with 1.
        ncol        (int): Number of colors in the final colormap - default is 256
    
    Output:
        A color map in matplotlib `LinearSegmentedColormap` object 
        '''
    
    rgb_list = [rgb_to_dec(hex_to_rgb(i)) for i in hex_list]
    if float_list:
        pass
    else:
        float_list = list(np.linspace(0,1,len(rgb_list)))
        
    cdict = dict()
    for num, col in enumerate(['red', 'green', 'blue']):
        col_list = [[float_list[i], rgb_list[i][num], rgb_list[i][num]] for i in range(len(float_list))]
        cdict[col] = col_list
    cmp = mcolors.LinearSegmentedColormap('my_cmp', segmentdata=cdict, N=ncol)
    return cmp



def grad_brite(xrgb,pcol=None,lite_0=0.0,lite_1=1.0,mid_lite=None,mid_spot=None, ncol=256):
    '''
    Remaps the color sequence xrgb (required) into a constant gradient of brightness 
        on the greyscale in 1 or 2 linear segments.
    Result is a color scale to use as a cmap in plotting.
    
    Required input: 
        xrgb     (list) = a list of rgb hex strings (form: 'Xrrggbb') giving sequence of colors;
                          it is assumed the first color at a position of 0.0 and the last at 1.0 for 
                          the interpolations described below.
    
    Optional arguments:
        pcol      (list) = a list of float positions on a scale 0.0-1.0 corresponding to colors in xrgb;
                           must be same length as xrgb (default is equally spaced)
                
        lite_0   (float) = brightness on scale 0.0-1.0 for first color in list (default 0.0)
        
        lite_1   (float) = brightness on scale 0.0-1.0 for last color in list (default 1.0)
        
        mid_lite (float) = if present, defines an intermediate point between 0.0 and 1.0 where
                           a third brightness level is defined. Two linear gradients of brightness 
                           are determined - one from 0.0 to mid_lite and one from mid_lite to 1.0.
                           Otherwise, a single linear gradient from positions 0.0 to 1.0 is defined.
        
        mid_spot (float) = position between 0.0 and 1.0 where mid_lite is placed;
                           ignored if mid_lite=None
                
        ncol       (int) = number of colors in the final colormap (default is 256)
        
    Returned results: 
        cmp     (object) = a ncol-stepped colormap object to use as a cmap in plotting
        
        cmp_r   (object) = The reversed version of cmp
    '''

    luma = [0.30, 0.59, 0.11]      # NTSC luma perception weightings for R, G, B
        
    # Nip potential problems with inputs
    lite_0 = np.max([0.0,np.min([1.0,lite_0])]) # Range is 0-1
    lite_1 = np.min([1.0,np.max([0.0,lite_1])]) # Range is 0-1
    ncol   = np.max([2,np.min([1024,ncol])])    # Range is 2-1024
    
    # Produce a preliminary color map, size ncol, from input colors - brightness will be adjusted below
    if pcol:
        if pcol[0] != 0.0:
            sys.exit("ERROR - First color position must be 0.0.")
        if pcol[-1] != 1.0:
            sys.exit("ERROR - Last color position must be 1.0.")
        if len(xrgb) != len(pcol):
            sys.exit("ERROR - List of positions is not the same length as list of colors.")
        else:
            rez = get_continuous_cmap(xrgb, float_list=pcol, ncol=ncol) # Produce color gradient - linear interp'd
    else:
        rez = get_continuous_cmap(xrgb, ncol=ncol) # Produce color gradient - default to equally spaced

    # Find the brightnesses of the originally supplied/requested colormap
    lite_rez = [np.dot(luma,rez(i)[:3]) for i in range(rez.N)]
        
    # Map out the target brightesses for the color map based on inputs
    if mid_lite:         # Two linear segments to interpolate
        if mid_spot:
            mid_x = int(np.rint(mid_spot*(ncol-1)))
            mid_x = np.max([np.min([mid_x,(ncol-2)]),1]) # Ensures midpoint is not same as first or last point
            lite = [lite_0 + (mid_lite-lite_0)*i/mid_x for i in range(mid_x)] + [mid_lite + (lite_1-mid_lite)*i/(ncol-mid_x-1) for i in range(ncol-mid_x)]
        else:
            sys.exit("ERROR - Location for intermediate brightness unspecified.")
    else:                # Only one segment
        lite = [lite_0 + (lite_1-lite_0)*i/(ncol-1) for i in range(ncol)] 

    # Rescaling of brightnesses at each point to produce perceptually uniform gradients
    fact = np.array(lite)/np.array(lite_rez)    
    fact3 = np.repeat(fact, repeats=3).reshape(ncol,3)   # Expand out to 3 array elements for array multiplication
    nurez3 = np.multiply(fact3,rez(range(ncol))[:,:3])   # Scaled values into a numpy array
    nurez3 = np.where(nurez3>1.0,1.0,np.where(nurez3<0.0,0.0,nurez3))
    nurez4 = np.ones((ncol,4)) ; nurez4[:,:-1] = nurez3  # Tack the alphas back on (all set to opaque)
    
    # Create a matplotlib colormap list for output
    cmp = mcolors.ListedColormap(nurez4)
    cmp_r = mcolors.ListedColormap(np.flip(nurez4,axis=0))
    
    return cmp,cmp_r


## Opening files


In [ ]:
#################################
# Open files

ddir = "/Volumes/SSD_8TB/CLASP/LES_runs2/" # Path to output from statistics_PSD_data.ipynb

# Open files
ds = xr.open_dataset(f"{ddir}fr2_stats2_PSD.nc4")

ds

## For publishable figures in paper

In [ ]:
nstats = 7                     # How many of the statistics to plot
l = list(ds.data_vars.keys())  # All the variables in the file
l2 = [l[x:x+nstats] for x in range(0,len(l),nstats)]
p01corr = 0.27   # 99% confidence level for correlations given sample size
p01corr = 0.20   # 95% confidence level for correlations given sample size

#sys.exit(0)

emrmin,emrmax = 1,5e4     # Log scale limits for plotting

box_props = dict(facecolor='#f0f5fa',edgecolor='#e0e0e0',alpha=0.5)
l_panel = ['a','b','c']

#####>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
# Loop through each variable's stats
for v in l2:
    eml_het,eml_hom,lv_het,lv_hom,emlr,corr,re = [ds[v[i]] for i in range(nstats)]
    var_name = v[6][3:]   # Extract the specific variable name from the re_* variabe
    print(var_name)

    eml_het = eml_het.assign_coords({"lt_hour":('hour',range(7,21))})
    eml_hom = eml_hom.assign_coords({"lt_hour":('hour',range(7,21))})
    corr = corr.assign_coords({"lt_hour":('hour',range(7,21))})

    if len(eml_het.shape) == 2: # The single-level fields - only one type of plot
        fig,ax = plt.subplots(1,3,figsize=(12,5),sharey='all')
        fig.subplots_adjust(wspace=0.07,hspace=0.02,left=0.06,right=0.99,bottom=0.22,top=0.94)  

        vmin,vmax = max(eml_het.min(),eml_het.max()*1e-10),eml_het.max()
        pc = ax[0].pcolormesh(eml_het.wavelength/4,eml_het.lt_hour,eml_het,
                              norm=mcolors.LogNorm(vmin=vmin,vmax=vmax),cmap='cubehelix')
        ax[1].pcolormesh(eml_hom.wavelength/4,eml_hom.lt_hour,eml_hom,
                         norm=mcolors.LogNorm(vmin=vmin,vmax=vmax),cmap='cubehelix')
        pd = ax[2].pcolormesh(eml_het.wavelength/4,eml_het.lt_hour,eml_het/eml_hom,
                              norm=mcolors.LogNorm(vmin=emrmin,vmax=emrmax),cmap='Spectral_r')
        # Add a significance contour
        ax[2].contour(eml_het.wavelength/4,eml_het.lt_hour,corr,[p01corr],colors='#80B0FF80',linewidths=5)
        ax[2].contour(eml_het.wavelength/4,eml_het.lt_hour,corr,[p01corr],colors='grey',linewidths=2.3)
        ax[2].contour(eml_het.wavelength/4,eml_het.lt_hour,corr,[p01corr],colors='w',linewidths=1)
        
        [ax[i].set_xscale('log') for i in range(3)]
        [ax[i].tick_params(axis='both',labelsize=12) for i in range(3)]
        ax[0].set_ylabel("Local Hour",fontsize=12)
        ax[1].set_xlabel("Wavelength [km]",fontsize=12)
        ax[0].set_title("HET",fontsize=16)
        ax[1].set_title("HOM",fontsize=16)
        ax[2].set_title("HET/HOM",fontsize=16)
        #plt.suptitle("Log Means; exp(mean(log(PSD)))", fontsize=12)
        cTL0 = fig.add_axes([0.11, 0.06, 0.50, 0.03]) 
        fig.colorbar(pc,extend='both',cax=cTL0,orientation='horizontal')
        cTL0.tick_params(labelsize=12)
        cTL1 = fig.add_axes([0.703, 0.06, 0.28, 0.03]) 
        fig.colorbar(pd,extend='neither',cax=cTL1,orientation='horizontal')
        cTL1.tick_params(labelsize=12)
        # fig.suptitle(f"{var_name}, [{eml_het.attrs['units']}]", fontsize='x-large')
        ax[0].set_yticks(range(7,21,1))
        ax[0].yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:02}"))
        
        # place a text box in upper left in axes coords
        [ax[i].text(0.55,20.6,f"{l_panel[i]})",va='bottom',ha='left',fontsize=16) for i in range(3)]
        
        fig.savefig(f"{var_name}_logmeans_time_wavelength.png",dpi=300)
        
        #sys.exit(0)
        
    else:  # The vertically-varying fields
        # Here we have two sets of plots...
        
        #####11111111111111111111111111111111111111111111111111111111111111
        ##### Wavenumber vs height; plot at midday
        hh = '18' # UTC hour for midday
        fig,ax = plt.subplots(1,3,figsize=(12,5),sharey='all')
        fig.subplots_adjust(wspace=0.07,hspace=0.02,left=0.06,right=0.99,bottom=0.22,top=0.94)  
        
        vmin,vmax = max(eml_het.sel(hour=hh)[:200].min(),eml_het.sel(hour=hh)[:200].max()*1e-10), eml_het.sel(hour=hh)[:200].max()
        pc = ax[0].pcolormesh(eml_het.wavelength/4,eml_het.p_levs/100,eml_het.sel(hour=hh),
                              norm=mcolors.LogNorm(vmin=vmin,vmax=vmax),cmap='cubehelix')
        ax[1].pcolormesh(eml_hom.wavelength/4,eml_hom.p_levs/100,eml_hom.sel(hour=hh),
                         norm=mcolors.LogNorm(vmin=vmin,vmax=vmax),cmap='cubehelix')
        pd = ax[2].pcolormesh(eml_het.wavelength/4,eml_het.p_levs/100,eml_het.sel(hour=hh)/eml_hom.sel(hour=hh),
                              norm=mcolors.LogNorm(vmin=emrmin,vmax=emrmax),cmap='Spectral_r')
        # Add a significance contour
        ax[2].contour(eml_het.wavelength/4,eml_het.p_levs/100,corr.sel(hour=hh),[p01corr],colors='#80B0FF80',linewidths=5)
        ax[2].contour(eml_het.wavelength/4,eml_het.p_levs/100,corr.sel(hour=hh),[p01corr],colors='grey',linewidths=2.3)
        ax[2].contour(eml_het.wavelength/4,eml_het.p_levs/100,corr.sel(hour=hh),[p01corr],colors='w',linewidths=1)
        
        [ax[i].set_xscale('log') for i in range(3)]
        [ax[i].tick_params(axis='both',labelsize=12) for i in range(3)]
        ax[0].set_ylabel("Pressure [hPa]",fontsize=12)
        ax[1].set_xlabel("Wavelength [km]",fontsize=12)
        ax[0].set_title("HET",fontsize=16)
        ax[1].set_title("HOM",fontsize=16)
        ax[2].set_title("HET/HOM",fontsize=16)
        ax[0].set_ylim(eml_het.p_levs[0]/100,401)
        cTL0 = fig.add_axes([0.11, 0.06, 0.50, 0.03]) 
        fig.colorbar(pc,extend='both',cax=cTL0,orientation='horizontal')
        cTL0.tick_params(labelsize=12)
        cTL1 = fig.add_axes([0.703, 0.06, 0.28, 0.03]) 
        fig.colorbar(pd,extend='neither',cax=cTL1,orientation='horizontal')
        cTL1.tick_params(labelsize=12)
        # fig.suptitle(f"{var_name}, [{eml_het.attrs['units']}]", fontsize='x-large')
        
        # place a text box in upper left in axes coords
        [ax[i].text(0.55,395,f"{l_panel[i]})",va='bottom',ha='left',fontsize=16) for i in range(3)]

        fig.savefig(f"{var_name}_logmeans_pressure_wavelength.png",dpi=300)
        
        #sys.exit(0)

        #####2222222222222222222222222222222222222222222222222222222222222222
        ##### Second figure has time / vertical slice panels at 3 wavelengths
        ww,wkm = [129,15,1], ["1","8","65"] # Corresponds to 1, 8.125 and 65 km  
        ww,wkm = [129,12,0], ["1","10","130"] # Corresponds to 1, 10 and 130 km
        ww,wkm = [129,5,0], ["1","22","130"] # Corresponds to 1, 16.25 and 130 km
        
        fig = plt.figure(constrained_layout=True,figsize=(12,5))
        plt.subplots_adjust(wspace=0.07,hspace=0.02,left=0.06,right=0.99,bottom=0.42,top=0.94)
        #subfigs = fig.subfigures(1,3)
        subfigs = fig.subfigures(2,3,height_ratios=[90,10])
        axLeft  = subfigs[0,0].subplots(1,3,sharey=True)
        axMid   = subfigs[0,1].subplots(1,3,sharey=True)
        axRight = subfigs[0,2].subplots(1,3,sharey=True)
        [subfigs[1,i].set_visible(False) for i in range(3)]
        vmin,vmax = max(eml_het.min(),eml_het.max()*1e-10), eml_het.max()

        # HET set
        pc = axLeft[0].pcolormesh(eml_het.lt_hour,eml_het.p_levs/100,eml_het[:200,:,ww[0]].T,
                                  norm=mcolors.LogNorm(vmin=vmin,vmax=vmax),cmap='cubehelix')
        axLeft[1].pcolormesh(eml_het.lt_hour,eml_het.p_levs/100,eml_het[:200,:,ww[1]].T,
                                  norm=mcolors.LogNorm(vmin=vmin,vmax=vmax),cmap='cubehelix')
        axLeft[2].pcolormesh(eml_het.lt_hour,eml_het.p_levs/100,eml_het[:200,:,ww[2]].T,
                                  norm=mcolors.LogNorm(vmin=vmin,vmax=vmax),cmap='cubehelix')   
        for i,w in enumerate(ww):  # Each slice
            axLeft[i].set_ylim(eml_het.p_levs[0]/100,401)
            axLeft[i].tick_params(axis='both',labelsize=12)
            axLeft[i].set_xticks([7,11,15,19])
            axLeft[i].set_title(f"{wkm[i]}km",fontsize=14)
            axLeft[i].xaxis.set_major_formatter(ticker.StrMethodFormatter("{x:02}"))

        axLeft[0].set_ylabel("Pressure [hPa]",fontsize=12)
        axLeft[1].set_xlabel("Local Hour",fontsize=12)
        axLeft[1].set_title(f"HET\n{wkm[1]}km",fontsize=14) # Override the one in the loop
        # place a text box in upper left in axes coords
        axLeft[0].text(2,396,f"{l_panel[0]})",va='bottom',ha='left',fontsize=16)
        
        # HOM set
        pc = axMid[0].pcolormesh(eml_hom.lt_hour,eml_hom.p_levs/100,eml_hom[:200,:,ww[0]].T,
                                  norm=mcolors.LogNorm(vmin=vmin,vmax=vmax),cmap='cubehelix')
        axMid[1].pcolormesh(eml_hom.lt_hour,eml_hom.p_levs/100,eml_hom[:200,:,ww[1]].T,
                                  norm=mcolors.LogNorm(vmin=vmin,vmax=vmax),cmap='cubehelix')
        axMid[2].pcolormesh(eml_hom.lt_hour,eml_hom.p_levs/100,eml_hom[:200,:,ww[2]].T,
                                  norm=mcolors.LogNorm(vmin=vmin,vmax=vmax),cmap='cubehelix')   
        for i,w in enumerate(ww):  # Each slice
            axMid[i].set_ylim(eml_het.p_levs[0]/100,401)
            axMid[i].tick_params(axis='both',labelsize=12)
            axMid[i].set_xticks([7,11,15,19])
            axMid[i].set_title(f"{wkm[i]}km",fontsize=14)
            axMid[i].xaxis.set_major_formatter(ticker.StrMethodFormatter("{x:02}"))
        axMid[1].set_xlabel("Local Hour",fontsize=12)
        axMid[1].set_title(f"HOM\n{wkm[1]}km",fontsize=14) # Override the one in the loop
        # place a text box in upper left in axes coords
        axMid[0].text(2,396,f"{l_panel[1]})",va='bottom',ha='left',fontsize=16)
        
        # HET/HOM
        pd = axRight[0].pcolormesh(eml_het.lt_hour,eml_het.p_levs/100,(eml_het[:200,:,ww[0]]/eml_hom[:200,:,ww[0]]).T,
                                   norm=mcolors.LogNorm(vmin=emrmin,vmax=emrmax),cmap='Spectral_r')
        axRight[1].pcolormesh(eml_het.lt_hour,eml_het.p_levs/100,eml_het[:200,:,ww[1]].T/eml_hom[:200,:,ww[1]].T,
                              norm=mcolors.LogNorm(vmin=emrmin,vmax=emrmax),cmap='Spectral_r')
        axRight[2].pcolormesh(eml_het.lt_hour,eml_het.p_levs/100,eml_het[:200,:,ww[2]].T/eml_hom[:200,:,ww[2]].T,
                                   norm=mcolors.LogNorm(vmin=emrmin,vmax=emrmax),cmap='Spectral_r')
        for i,w in enumerate(ww):  # Each slice
            axRight[i].set_ylim(eml_het.p_levs[0]/100,401)
            axRight[i].tick_params(axis='both',labelsize=12)
            axRight[i].set_xticks([7,11,15,19])
            axRight[i].set_title(f"{wkm[i]}km",fontsize=14)
            axRight[i].xaxis.set_major_formatter(ticker.StrMethodFormatter("{x:02}"))
            # Add a significance contour
            axRight[i].contour(eml_het.lt_hour,eml_het.p_levs/100,corr[:200,:,ww[i]].T,[p01corr],colors='#80B0FF80',linewidths=5)
            axRight[i].contour(eml_het.lt_hour,eml_het.p_levs/100,corr[:200,:,ww[i]].T,[p01corr],colors='grey',linewidths=2.3)
            axRight[i].contour(eml_het.lt_hour,eml_het.p_levs/100,corr[:200,:,ww[i]].T,[p01corr],colors='w',linewidths=1)
        axRight[1].set_xlabel("Local Hour",fontsize=12)
        axRight[1].set_title(f"HET/HOM\n{wkm[1]}km",fontsize=14) # Override the one in the loop
        # place a text box in upper left in axes coords
        axRight[0].text(2,396,f"{l_panel[2]})",va='bottom',ha='left',fontsize=16)
        
        cTL0 = fig.add_axes([0.11, 0.06, 0.50, 0.03],zorder=3) 
        fig.colorbar(pc,extend='both',cax=cTL0,orientation='horizontal')
        cTL0.tick_params(labelsize=12)
        cTL1 = fig.add_axes([0.723, 0.06, 0.26, 0.03],zorder=3) 
        fig.colorbar(pd,extend='neither',cax=cTL1,orientation='horizontal')
        cTL1.tick_params(labelsize=12)
        # fig.suptitle(f"{var_name}, [{eml_het.attrs['units']}]", fontsize='x-large')

        fig.savefig(f"{var_name}_logmeans_pressure_time.png",dpi=300)

        #sys.exit(0)
        
        #fig.clear()
    #sys.exit(0)

## For figures to study results

Had found in previous versions that:
* Correlation tells everything you need to know - relative entropy is a bit of a wasted effort
* The variances are not all that helpful, except to confirm what comes from synoptic (and surface) situations vs what is robust internal dynamics of the system.